# PULSE trace dashboard v0 demo

This notebook visualises the shadow-only memory / trace artefacts
produced by the PULSE EPF + paradox pipelines:

- `decision_history_v0.json`
- `paradox_history_v0.json`
- `trace_dashboard_v0.json`


## Setup

This assumes you have already run the shadow tools to produce the JSON
artefacts under `PULSE_safe_pack_v0/artifacts/`, and that this notebook
lives in `PULSE_safe_pack_v0/examples/`.

If your layout is different, adjust `ARTIFACT_DIR` below.


In [ ]:
import json
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

# Where the JSON artefacts live relative to this notebook.
ARTIFACT_DIR = Path("../artifacts")

decision_history_path = ARTIFACT_DIR / "decision_history_v0.json"
paradox_history_path = ARTIFACT_DIR / "paradox_history_v0.json"
trace_dashboard_path = ARTIFACT_DIR / "trace_dashboard_v0.json"

print("Decision history:", decision_history_path)
print("Paradox history:", paradox_history_path)
print("Trace dashboard:", trace_dashboard_path)


In [ ]:
def _load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

decision_history = _load_json(decision_history_path)
paradox_history = _load_json(paradox_history_path)
trace_dashboard = _load_json(trace_dashboard_path)

print("Keys in trace_dashboard:", list(trace_dashboard.keys()))


In [ ]:
def _extract_runs(decision_history_obj):
    if isinstance(decision_history_obj, dict) and isinstance(
        decision_history_obj.get("runs"), list
    ):
        return decision_history_obj["runs"]
    elif isinstance(decision_history_obj, list):
        return decision_history_obj
    else:
        return [decision_history_obj]

def _extract_axes(paradox_history_obj):
    if isinstance(paradox_history_obj, dict) and isinstance(
        paradox_history_obj.get("axes"), list
    ):
        return paradox_history_obj["axes"]
    elif isinstance(paradox_history_obj, list):
        return paradox_history_obj
    else:
        return []

runs = _extract_runs(decision_history)
axes = _extract_axes(paradox_history)

runs_df = pd.DataFrame(runs)
axes_df = pd.DataFrame(axes)

runs_df.tail()


In [ ]:
axes_df.head()


In [ ]:
# Plot instability per run (if available)
plot_df = runs_df.copy()

if "run_id" in plot_df.columns:
    plot_df = plot_df.sort_values("run_id")

instability_col = None
for candidate in ["instability", "instability_score"]:
    if candidate in plot_df.columns:
        instability_col = candidate
        break

if instability_col is None:
    print("No instability column found in decision history.")
else:
    plt.figure(figsize=(8, 4))
    plt.plot(plot_df["run_id"], plot_df[instability_col], marker="o")
    plt.xticks(rotation=45, ha="right")
    plt.ylabel(instability_col)
    plt.title("Instability by run")
    plt.tight_layout()


In [ ]:
# Plot paradox axes by severity / dominance (if available)
if axes_df.empty:
    print("No axes found in paradox history.")
else:
    severity_order = {"LOW": 1, "MEDIUM": 2, "HIGH": 3, "CRITICAL": 4}
    axes_plot_df = axes_df.copy()
    if "severity" in axes_plot_df.columns:
        axes_plot_df["severity_rank"] = axes_plot_df["severity"].map(severity_order).fillna(0)
    else:
        axes_plot_df["severity_rank"] = 0

    if "times_dominant" in axes_plot_df.columns:
        axes_plot_df = axes_plot_df.sort_values(
            ["severity_rank", "times_dominant"], ascending=False
        )
    else:
        axes_plot_df = axes_plot_df.sort_values("severity_rank", ascending=False)

    if "axis_id" not in axes_plot_df.columns:
        print("No axis_id column found in paradox history.")
    else:
        plt.figure(figsize=(8, 4))
        plt.bar(axes_plot_df["axis_id"], axes_plot_df["severity_rank"])
        plt.xticks(rotation=45, ha="right")
        plt.ylabel("severity_rank")
        plt.title("Paradox axes – severity (ranked)")
        plt.tight_layout()


In [ ]:
from pprint import pprint

print("Decision overview:")
pprint(trace_dashboard.get("decision_overview", {}))

print("")
print("Paradox overview:")
pprint(trace_dashboard.get("paradox_overview", {}))


## Next steps

This notebook is intentionally minimal. It is meant as a starting point
for building richer dashboards on top of the trace artefacts.

Ideas for extensions:

- join the trace data with Stability Map states / transitions,
- add filters by decision type or paradox zone,
- plot EPF-related fields once those are logged into history.
